<a id="section_0"></a>
# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
from sklearn.metrics import mean_squared_error

In [4]:
from numpy.random import RandomState
state = RandomState(12345) 

Введем базовые величины для расчета проекта.

In [5]:
budget_per_region = 10_000_000_000 # бюджет на разработку региона, рублей
revenue_per_kbarrel = 450_000 # доход с 1 тыс. барелей, рублей
n_boreholes = 200 # количество рабочих скважин на регион
n_locations = 500 # количество пробных бурений на регион

<a id="section_1"></a>
## Загрузка и подготовка данных

Загрузим данные, предусмотрим различные способы расположения файлов.

In [6]:
try:
    geo_data_0 = pd.read_csv('geo_data_0.csv')
    geo_data_1 = pd.read_csv('geo_data_1.csv')
    geo_data_2 = pd.read_csv('geo_data_2.csv')
except:
    geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
    geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
    geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

Напишем функцию, автоматизирующую процесс проверки данных.

In [7]:
def check(df):
    
    print(f'строк={df.shape[0]}, столбцов={df.shape[1]}')
    print()
    print(f'дублей: {df.duplicated().sum()}')
    print()
    print(df.head())
    print()
    print(df.info())
    print()
    print(df.describe(include='all'))
    print()

Проверим данные по каждому региону.

In [8]:
check(geo_data_0)

строк=100000, столбцов=5

дублей: 0

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

            id             f0             f1             f2        product
count   100000  100000.000000  100000.000000  100000.000000  100000.000000
unique   99990            NaN            NaN            NaN       

Данные читаемы, пропусков нет, формат данных соответствует ожидаемому, дубли отсутствуют.

In [9]:
check(geo_data_1)

строк=100000, столбцов=5

дублей: 0

      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

            id             f0             f1             f2        product
count   100000  100000.000000  100000.000000  100000.000000  100000.000000
unique   99996            NaN            NaN          

In [10]:
check(geo_data_2)

строк=100000, столбцов=5

дублей: 0

      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

            id             f0             f1             f2        product
count   100000  100000.000000  100000.000000  100000.000000  100000.000000
unique   99996            NaN            NaN            NaN       

Для всех регионов: данные читаемы, пропусков нет, формат данных соответствует ожидаемому, дубли отсутствуют. 

Можем приступить к подготовке данных.

Удалим столбцы 'id' как не несущие значимой информации.

In [11]:
geo_data_0 = geo_data_0.drop(['id'], axis=1)
geo_data_1 = geo_data_1.drop(['id'], axis=1)
geo_data_2 = geo_data_2.drop(['id'], axis=1)

Отделим целевой признак и разобьем данные на обучающую и валидационную выборки в соотношении 75:25. Проделаем эти манипуляции для каждого региона.

In [12]:
features_0 = geo_data_0.drop(['product'], axis=1)
target_0 = geo_data_0['product']
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=.25, random_state=12345)

print(features_train_0.shape)
print(features_valid_0.shape)
print(target_train_0.shape)
print(target_valid_0.shape)

(75000, 3)
(25000, 3)
(75000,)
(25000,)


In [13]:
features_1 = geo_data_1.drop(['product'], axis=1)
target_1 = geo_data_1['product']
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=.25, random_state=12345)

print(features_train_1.shape)
print(features_valid_1.shape)
print(target_train_1.shape)
print(target_valid_1.shape)

(75000, 3)
(25000, 3)
(75000,)
(25000,)


In [14]:
features_2 = geo_data_2.drop(['product'], axis=1)
target_2 = geo_data_2['product']
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=.25, random_state=12345)

print(features_train_2.shape)
print(features_valid_2.shape)
print(target_train_2.shape)
print(target_valid_2.shape)

(75000, 3)
(25000, 3)
(75000,)
(25000,)


Масштабирум обучающие выборки с применением StandardScaler. Также для каждого региона.

In [15]:
scaler_0 = StandardScaler()
scaler_0.fit(features_train_0)

features_train_0 = scaler_0.transform(features_train_0)
features_valid_0 = scaler_0.transform(features_valid_0)

In [16]:
scaler_1 = StandardScaler()
scaler_1.fit(features_train_1)

features_train_1 = scaler_1.transform(features_train_1)
features_valid_1 = scaler_1.transform(features_valid_1)

In [17]:
scaler_2 = StandardScaler()
scaler_2.fit(features_train_2)

features_train_2 = scaler_2.transform(features_train_2)
features_valid_2 = scaler_2.transform(features_valid_2)

### Вывод

Данные загружены и проверены.

Отделены данные с целевым признаком. Произведено разбиение на обучающие и валидационные выборки. Данные масштабированы.

Загрузка и подготовка данных проведены успешно. Можно приступать к обучению моделей.

[В начало раздела 1](#section_1)

<a id="section_2"></a>
## Обучение и проверка модели

Создадим модели для каждого региона. В качестве обучающего алгоритма выберем LinearRegression.

In [18]:
model_0 = LinearRegression(n_jobs=-1)
model_1 = LinearRegression(n_jobs=-1)
model_2 = LinearRegression(n_jobs=-1)

Обучим модели на соответствующих выборках.
<a id="step_1"></a>

In [19]:
model_0.fit(features_train_0, target_train_0)
model_1.fit(features_train_1, target_train_1)
model_2.fit(features_train_2, target_train_2)

LinearRegression(n_jobs=-1)

Сделаем предсказания на валидационных данных.
<a id="step_2"></a>

In [20]:
predict_0 = model_0.predict(features_valid_0)
predict_1 = model_1.predict(features_valid_1)
predict_2 = model_2.predict(features_valid_2)

Сохраним предсказания и правильные ответы на валидационной выборке.
<a id="step_3"></a>

In [21]:
target_predict_0 = pd.DataFrame(target_valid_0)
target_predict_0['predict'] = predict_0
target_predict_1 = pd.DataFrame(target_valid_1)
target_predict_1['predict'] = predict_1
target_predict_2 = pd.DataFrame(target_valid_2)
target_predict_2['predict'] = predict_2

Оценим качество предсказания моделей по метрике R2.

In [22]:
score_0 = model_0.score(features_valid_0, target_valid_0)
score_1 = model_1.score(features_valid_1, target_valid_1)
score_2 = model_2.score(features_valid_2, target_valid_2)
print(f'score_0 = {score_0:.3f}, score_1 = {score_1:.3f}, score_2 = {score_2:.3f}')

score_0 = 0.280, score_1 = 1.000, score_2 = 0.205


Значения метрики R2 для всех моделей больше 0, что говорит нам о том, проверка на адекватность пройдена. Уровни R2 для моделей по регионам 0 и 2 - 0.28 и 0.21 соответственно - говорят нам о невысоком качестве предсказаний. Уровень R2 для модели по региону 1 - 1.000 - говорит нам о идеальном качестве предсказаний.

Выведем средний запас предсказанного сырья и RMSE модели.

In [23]:
rmse_0 = mean_squared_error(target_valid_0, predict_0) ** .5
rmse_1 = mean_squared_error(target_valid_1, predict_1) ** .5
rmse_2 = mean_squared_error(target_valid_2, predict_2) ** .5

In [24]:
print(f'регион_0: средний запас предсказанного сырья={predict_0.mean():.2f}, RMSE={rmse_0:.2f}')
print(f'регион_1: средний запас предсказанного сырья={predict_1.mean():.2f}, RMSE={rmse_1:.2f}')
print(f'регион_2: средний запас предсказанного сырья={predict_2.mean():.2f}, RMSE={rmse_2:.2f}')

регион_0: средний запас предсказанного сырья=92.59, RMSE=37.58
регион_1: средний запас предсказанного сырья=68.73, RMSE=0.89
регион_2: средний запас предсказанного сырья=94.97, RMSE=40.03


### Вывод

Нами были созданы, обучены и оценены модели для каждого региона.

Получены значения метрики R2 для каждого региона:
- score_0 = 0.280
- score_1 = 1.000
- score_2 = 0.205

Также были сделаны предсказания и получены следующие результаты:
- регион_0: средний запас предсказанного сырья=92.59, RMSE=37.58
- регион_1: средний запас предсказанного сырья=68.73, RMSE=0.89
- регион_2: средний запас предсказанного сырья=94.97, RMSE=40.03

Полученные цифры говорят о следующем:
- средний запас предсказанного сырья для регионов 0 и 2 гораздо выше, чем для региона 1,
- в то же время точность предсказаний для регионов 0 и 2 гораздо ниже, чем для региона 1.

Результаты работы моделей сохранены для дальнейшего изучения.

[В начало раздела 2](#section_2)

<a id="section_3"></a>
## Подготовка к расчёту прибыли

Сохраним ключевые значения для расчётов в отдельных переменных.

In [25]:
#  объём сырья для безубыточной разработки нового региона
self_repayment_value_per_region = budget_per_region / revenue_per_kbarrel

#  объём сырья для безубыточной разработки новой скважины
self_repayment_value_per_borehole = self_repayment_value_per_region / n_boreholes

Выведем объём сырья для безубыточной разработки новой скважины.

In [26]:
print(f'Объём сырья для безубыточной разработки скважины: {self_repayment_value_per_borehole:.2f} (тыс. баррелей)')

Объём сырья для безубыточной разработки скважины: 111.11 (тыс. баррелей)


Сравним полученный объём сырья со средним запасом в каждом регионе.

На предыдущем этапе мы получили следующие значения на предсказанных данных:
- регион_0: средний запас предсказанного сырья=92.59
- регион_1: средний запас предсказанного сырья=68.73
- регион_2: средний запас предсказанного сырья=94.97

Видим, что без предварительного отбора лучших скважин, бизнес будет убыточным.

Посмотрим, каковы будут значения для фактических данных.

In [27]:
print(f'регион_0: средний запас предсказанного сырья={target_valid_0.mean():.2f}')
print(f'регион_1: средний запас предсказанного сырья={target_valid_1.mean():.2f}')
print(f'регион_2: средний запас предсказанного сырья={target_valid_2.mean():.2f}')

регион_0: средний запас предсказанного сырья=92.08
регион_1: средний запас предсказанного сырья=68.72
регион_2: средний запас предсказанного сырья=94.88


Необходимость предварительного отбора лучших скважин подтверждается и на реальных данных.

Напишим функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели.

In [28]:
def profit(df):
    
    # отбор скважин с максимальными предсказанными запасами 
    df_separate = df.sort_values('predict').tail(n_boreholes) 
    
    # прибыль = сумма запасов в отобранных скважинах * цена 1 тыс баррелей - затраты на разработку региона
    profit = df_separate['product'].sum() * revenue_per_kbarrel - budget_per_region 
    
    return profit

### Вывод

На этом этапе исследования нами были введены необходимые метрики:
- объём сырья для безубыточной разработки нового региона
- объём сырья для безубыточной разработки новой скважины

Вычислен объём сырья для безубыточной разработки скважины: 
- 111.11 (тыс. баррелей)

Выявлена необходимость предварительного отбора лучших скважин.

Написана функция для расчёта прибыли по выбранным скважинам и предсказаниям модели.

[В начало раздела 3](#section_3)

<a id="section_4"></a>
## Расчёт прибыли и рисков 

Посчитаем риски и прибыль для каждого региона.

Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли для каждого региона.

In [29]:
profit_0 = []

for i in range(1000):
    
    df = target_predict_0.sample(n=n_locations, replace=True, random_state=state)
    profit_0.append(profit(df))

In [30]:
profit_1 = []

for i in range(1000):
    
    df = target_predict_1.sample(n=n_locations, replace=True, random_state=state)
    profit_1.append(profit(df))

In [31]:
profit_2 = []

for i in range(1000):
    
    df = target_predict_2.sample(n=n_locations, replace=True, random_state=state)
    profit_2.append(profit(df))

Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков.

Напишем функцию, автоматизирующую данный процесс.

In [32]:
def check_profit(profit_list):
    
    mean = pd.Series(profit_list).mean()
    lower = pd.Series(profit_list).quantile(q=0.025)
    upper = pd.Series(profit_list).quantile(q=0.975)
    print(f'средняя прибыль={mean / 10 ** 6:.1f} млн руб')
    print(f'95%-й доверительный интервал: {lower / 10 ** 6:.1f} - {upper / 10 ** 6:.1f} млн руб')
    
    for quant in np.arange(0, 1, 0.005):
        profit = pd.Series(profit_list).quantile(q=quant) 
        if profit >= 0:
            print(f"риск убытков = {quant:.1%}")
            break

Выведем интересующие показатели для каждого региона.

In [33]:
check_profit(profit_0)

средняя прибыль=396.2 млн руб
95%-й доверительный интервал: -111.2 - 909.8 млн руб
риск убытков = 7.0%


In [34]:
check_profit(profit_1)

средняя прибыль=461.2 млн руб
95%-й доверительный интервал: 78.1 - 863.0 млн руб
риск убытков = 1.0%


In [35]:
check_profit(profit_2)

средняя прибыль=393.0 млн руб
95%-й доверительный интервал: -112.2 - 934.6 млн руб
риск убытков = 6.5%


### Вывод

Требуемому уровню риска в 2.5% соответствует лишь один из регионов: регион_1 - риск убытков = 1.0%.

Средняя прогнозируемая прибыль в этом регионе также наибольшая: 461.2 млн руб.

Также отметим, что качество предсказаний модели по этому региону практически идеально.

Учитывая изложенное рекомендуется выбрать для бурения новых скважин регион_1.

[В начало раздела 4](#section_4)

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован
    
[В начало](#section_0)